<a href="https://colab.research.google.com/github/Kieil/Titanic-ML/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC


# New Section

In [ ]:
import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)

/content


In [ ]:
from typing_extensions import assert_type
#getting files
gender = pd.read_csv('gender_submission.csv')
X_test = pd.read_csv('test.csv')
df = pd.read_csv('train.csv')


df['train_test'] = 1
X_test['train_test'] =0
X_test['Survived'] = np.NaN
all_data = pd.concat([df, X_test])

In [ ]:
#check nulls values in data
missing_values = df.isnull().sum()
print(missing_values)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
train_test       0
dtype: int64


In [ ]:
#filling nulls with 0/ S / mean
all_data['Cabin'] = all_data['Cabin'].fillna(0)
all_data['Embarked'] = all_data['Embarked'].fillna('S')
all_data['Age'] = all_data['Age'].fillna(X_test['Age'].mean())



In [ ]:
#changing type to string
all_data['Cabin'] =all_data['Cabin'].astype(str)

In [ ]:
#data modelling
# binary sex , ports to numbers, titles to numbers, splitting cabin
all_data['Sex_binary'] = all_data['Sex'].apply(lambda x:1 if x=='male' else 0)
all_data['Port'] = all_data['Embarked'].apply(lambda x:1 if x=='S' else(2 if x=='Q' else 0))
all_data['Title'] = all_data['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip())
all_data['Bo_Title'] = all_data['Title'].apply(lambda x:1 if x=='Mr' else(2 if x=='Miss' else 0))
all_data['Cabin_splitted'] = all_data['Cabin'].apply(lambda x:x.split(' ')[0] if x else 0)



In [ ]:
#exporting level of ship from cabin
import string

all_data['Cabin_splitted'] = all_data['Cabin_splitted'].fillna(0)
def letter_to_number(letter):
    if letter and letter.isalpha():
        return string.ascii_uppercase.index(letter.upper()) + 1
    else:
        return 0

level = all_data['Cabin_splitted'].apply(lambda x: letter_to_number(x[0] if x else 0))

all_data['level'] = level


In [ ]:
X_train1 = all_data[all_data.train_test == 1].drop(['train_test'], axis =1)
X_train = X_train1[['PassengerId', 'Age', 'SibSp', 'Parch','Sex_binary', 'Fare','Pclass','Port','Bo_Title','level']].astype(str)
X_test1 = all_data[all_data.train_test == 0].drop(['train_test'], axis =1)
X_test = X_test1[['PassengerId', 'Age', 'SibSp', 'Parch','Sex_binary','Fare', 'Pclass','Port','Bo_Title','level']].astype(str)


In [ ]:
#making list of features
Y_train = X_train1['Survived'] == 1
y_test = X_test1['Survived']

In [ ]:
#importing ml models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb

In [ ]:
Y_train

,Survived
0,False
1,True
2,True
3,True
4,False
...,...
886,False
887,True
888,False
889,True


In [ ]:
model1 = HistGradientBoostingClassifier()
model1.fit(X_train, Y_train)

HistGradientBoostingClassifier()

In [ ]:
model2 = HistGradientBoostingClassifier()
model2.fit(X_train, Y_train)

HistGradientBoostingClassifier()

In [ ]:
model3 = DecisionTreeClassifier()
model3.fit(X_train, Y_train)

DecisionTreeClassifier()

In [ ]:
model4 = DecisionTreeClassifier()
model4.fit(X_train, Y_train)

DecisionTreeClassifier()

In [ ]:
#model7 = xgb.XGBClassifier()
#Y_train_males = Y_train_males.astype(int) # Convert the target variable to integers
#model7.fit(X_train_males, Y_train_males)

In [ ]:
#model8 = xgb.XGBClassifier()
#Y_train_females = Y_train_females.astype(int)
#model8.fit(X_train_females, Y_train_females)

In [ ]:
ensemble = VotingClassifier(estimators=[('hg1', model1), ('hg2', model2),('dt1', model3), ('dt2', model4)], voting='hard')

In [ ]:
ensemble.fit(X_train, Y_train )

VotingClassifier(estimators=[('hg1', HistGradientBoostingClassifier()),
                             ('hg2', HistGradientBoostingClassifier()),
                             ('dt1', DecisionTreeClassifier()),
                             ('dt2', DecisionTreeClassifier())])

In [ ]:
y_pred = ensemble.predict(X_test)
y_pred.astype(int)

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
#from sklearn.metrics import accuracy_score

# Calculate predictions on the test set using the ensemble
#y_pred = ensemble.predict(X_train)

# Calculate accuracy using the true labels (Y_test) and predicted labels
#accuracy = accuracy_score(Y_train, y_pred)  # Replace X_test with Y_test
#print("Test Accuracy:", accuracy)  # Changed from "Training Accuracy"

In [ ]:
submission = pd.DataFrame({
    'PassengerId': X_test['PassengerId'],
    'Survived': y_pred.astype(int)  # Convert predictions to integers
})

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
